### Simple Feedback System

The goal of this notebook is to gently introduce a variant of the [CAL theorem](https://arxiv.org/abs/2109.07771) by Lee et al, which is more fine-grained than the original and can be more easily applied to the [Lingua Franca](https://github.com/lf-lang/lingua-franca) coordination language.

Consider the following feedback system.

<img src="../img/StockExchange.svg" alt="drawing" width="600"/>

#### Max-Plus Operators

In [ ]:
# Max-plus functions from https://github.com/msnhdyt/Max-Plus-Algebra
import numpy as np

# function for calculating A oplus B
def oplus(A, B):
  if not type(A) is np.ndarray or A.ndim != 2:
    raise TypeError("A must be a 2d numpy array")
  if not type(B) is np.ndarray or B.ndim != 2:
    raise TypeError("B must be a 2d numpy array")
  if A.shape != B.shape:
    raise TypeError("A and B must have the same shape: A.shape={}, B.shape={}".format(A.shape, B.shape))
  result = np.zeros(A.shape)
  for i, r in enumerate(A):
    for j, c in enumerate(r):
      result[i][j] = max(A[i][j], B[i][j])
  return result

# Function for calculating A otimes B, i.e., matrix multiplication
def otimes(A, B):
  if not type(A) is np.ndarray or A.ndim != 2:
    raise TypeError("A must be a 2d numpy array")
  if not type(B) is np.ndarray or B.ndim != 2:
    raise TypeError("B must be a 2d numpy array")
  if A.shape[1] != B.shape[0]:
    raise TypeError("A's 2nd dimension does not match B's 1st dimension: A.shape={}, B.shape={}".format(A.shape, B.shape))

  if A.shape == (1,1) or B.shape == (1,1):
    result = A + B
    return result
  else:
    result = np.zeros([A.shape[0], B.shape[1]])
    B = B.T
    for i, row_a in enumerate(A):
      for j, row_b in enumerate(B):
        result[i][j] = odot(row_a, row_b)
    return result
  
# Function for taking the max-plus dot product of two row vectors
def odot(a, b):
  assert a.ndim == b.ndim == 1, "Vector a and b must be 1D arrays"
  assert len(a) == len(b), "Vector a and b must have the same shape"
  result = np.array([float('-inf')] * len(a))
  for i in range(0, len(a)):
    result[i] = a[i] + b[i]
  return max(result)

# function for calculating A^n in otimes
def pow_otimes(A, n):
  if not type(A) is np.ndarray or A.ndim != 2:
    raise TypeError("the function only receive 2d numpy array")
  result = A
  for i in range(n-1):
    result = otimes(result, A)
  return result

# function for calculating trace(A)
def trace(A):
  if not type(A) is np.ndarray or A.ndim != 2:
    raise TypeError("the function only receive 2d numpy array")
  return max(A.diagonal())

# function for calculating A*
def star(A):
  if not type(A) is np.ndarray or A.ndim != 2:
    raise TypeError("the function only receive 2d numpy array")
  
  result = np.zeros(A.shape)
  # initialize result with E
  for i in range(len(A)):
    for j in range(len(A)):
      if i == j:
        result[i][j] = 0
      else:
        result[i][j] = float('-inf')

  for i in range(1,len(A)):
    result = oplus(result, pow_otimes(A,i))
  
  return result

# function for calculating A^+
def plus(A):
  if not type(A) is np.ndarray or A.ndim != 2:
    raise TypeError("the function only receive 2d numpy array")
  
  result = A

  for i in range(1,len(A)+1):
    result = oplus(result, pow_otimes(A,i))
  
  return result

#### Constants and variables

In [ ]:
# Negative infinity to denote nothing has happened yet
# epsilon = empty element = -inf
eps = float('-inf')
# Positive infinity to denote the exhaustion of future events
inf = float('inf')

# Return an epsilon-filled row vector with the same shape as the input vector.
def reset(a):
    return np.array([[eps] * a.shape[1]])

# Create vectors. These are row vectors, i.e., shape = 1 x n.
x_p = np.array([[eps] * 2])   # Physical firing times in the k+1-th iteration
x   = np.array([[eps] * 2])   # Physical firing times in the k-th iteration
u   = np.array([[eps] * 1])   # Logical scheduling times of physical actions (which is also physical time)

# Indices
idx_c = 0 # Reaction in reactor Client
idx_e = 1 # Reaction in reactor Exchange

# Setting execution times
e = np.array([[1, 1]])   # Execution times

#### M and N matrix

In [ ]:
# Evolution matrix
M = np.array([
    [e[0][idx_c], eps], 
    [e[0][idx_c]+e[0][idx_c], e[0][idx_e]],
])

# Control coefficient matrix
N = np.array([
    [0],
    [e[0][idx_c]],
])

#### Helper functions

In [ ]:
def step(x, u):
    assert x.shape[0] == 1, "x is not a row vector: x.shape={0}".format(x.shape)
    assert u.shape[0] == 1, "u is not a row vector: u.shape={0}".format(u.shape)
    Mx = otimes(M, x.T)
    Nu = otimes(N, u.T)
    # print("Mx:{}".format(Mx))
    # print("Nu:{}".format(Nu))
    # Returns a row vector to make iterations easy.
    return oplus(Mx, Nu).T

In [ ]:
def report_stats(k, x, u):
    next_logical_time = np.max(u)
    lag_k = x - next_logical_time # lag
    print("Tag index k={k}".format(k=k))
    print("u(k)= {0}".format(u))
    # print("next logical time = {0}".format(next_logical_time))
    print("x(k) = earliest possible firing times = {0}".format(x))
    print("lag(k) = {0}".format(lag_k))

#### A function for modeling user orders

In [ ]:
# Initial firing of actions
action_init_offset = 0
u[0][c] = action_init_offset

# Set the period of the actions 
# (Either same or one is -inf. See NOTE above)
# NOTE 2: eigenvalue(M) = smallest minimum spacing of the physical action to make the systme feasible
action_period = 1 # If actions scheduled simultaneously, use 0.999 to see divergence. Use inf if no future events are scheduled.

# Interation count
k = 0

# Reset x to eps.
x = reset(x)

#### Evolve the system

In [ ]:
# Step the system from k=0 to k=1
x = step(x, u)
k += 1

# Statistics
report_stats(k, x, u)

In [ ]:
# Bump the next physical action firing times
u[0][c] = u[0][c] + action_period

# Step the system from k to k+1
x = step(x, u)
k += 1

# Statistics
report_stats(k, x, u)